In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
#Importing dependencies
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
import time
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

In [3]:
features=["Pclass", "Sex","Age", "SibSp", "Parch","Embarked","Cabin Index","Fare"]

In [4]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
def testNa (df,features=features):
    for feat in features : 
        print(feat," : NANs : ", sum(df[feat].isna()))

In [6]:
def fillCabin(df):
    df = df.copy()
    df.loc[df['Cabin'].notnull(), 'Cabin Index'] = df['Cabin'].str[0]
    df.fillna({"Cabin Index":"Unknown"},inplace=True)
    return df

In [7]:
def FillN (orig_df, cols):
    df = orig_df.copy()
    for col in cols:
        s = df[col].value_counts(normalize=True)
        print(s)
        missing = df[col].isnull()
        df.loc[missing,col] = np.random.choice(s.index, size=len(df[missing]),p=s.values)
    return df

In [8]:
df = train_data.copy()
new_train_df = FillN(df, ["Age","Embarked","Fare"])
new_train_df = fillCabin(new_train_df)

Age
24.00    0.042017
22.00    0.037815
18.00    0.036415
19.00    0.035014
28.00    0.035014
           ...   
36.50    0.001401
55.50    0.001401
0.92     0.001401
23.50    0.001401
74.00    0.001401
Name: proportion, Length: 88, dtype: float64
Embarked
S    0.724409
C    0.188976
Q    0.086614
Name: proportion, dtype: float64
Fare
8.0500     0.048260
13.0000    0.047138
7.8958     0.042649
7.7500     0.038159
26.0000    0.034792
             ...   
35.0000    0.001122
28.5000    0.001122
6.2375     0.001122
14.0000    0.001122
10.5167    0.001122
Name: proportion, Length: 248, dtype: float64


In [9]:
train_data.Cabin.unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [10]:
new_train_df["Cabin Index"].unique()

array(['Unknown', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [11]:
new_train_df.index[new_train_df['Cabin']=="T"].tolist()


[339]

In [12]:
new_train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin Index
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Unknown
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Unknown
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Unknown


In [13]:
testNa(new_train_df)

Pclass  : NANs :  0
Sex  : NANs :  0
Age  : NANs :  0
SibSp  : NANs :  0
Parch  : NANs :  0
Embarked  : NANs :  0
Cabin Index  : NANs :  0
Fare  : NANs :  0


In [14]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [15]:
test_df = test_data.copy()
new_test_df = FillN(test_df, ["Age","Embarked","Fare"])
new_test_df = fillCabin(new_test_df)

Age
21.0    0.051205
24.0    0.051205
22.0    0.048193
30.0    0.045181
18.0    0.039157
          ...   
76.0    0.003012
28.5    0.003012
22.5    0.003012
62.0    0.003012
38.5    0.003012
Name: proportion, Length: 79, dtype: float64
Embarked
S    0.645933
C    0.244019
Q    0.110048
Name: proportion, dtype: float64
Fare
7.7500     0.050360
26.0000    0.045564
13.0000    0.040767
8.0500     0.040767
7.8958     0.026379
             ...   
7.8208     0.002398
8.5167     0.002398
78.8500    0.002398
52.0000    0.002398
22.3583    0.002398
Name: proportion, Length: 169, dtype: float64


In [16]:
new_train_df.shape

(891, 13)

In [17]:
new_test_df.shape

(418, 12)

In [18]:
women = new_train_df.loc[new_train_df.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

men = new_train_df.loc[new_train_df.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of women who survived: 0.7420382165605095
% of men who survived: 0.18890814558058924


In [19]:
y = new_train_df.copy()["Survived"]

X = pd.get_dummies(new_train_df.copy()[features])
X_test = pd.get_dummies(new_test_df.copy()[features])

In [20]:
X.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin Index_A,Cabin Index_B,Cabin Index_C,Cabin Index_D,Cabin Index_E,Cabin Index_F,Cabin Index_G,Cabin Index_T,Cabin Index_Unknown
0,3,22.0,1,0,7.2500,False,True,False,False,True,False,False,False,False,False,False,False,False,True
1,1,38.0,1,0,71.2833,True,False,True,False,False,False,False,True,False,False,False,False,False,False
2,3,26.0,0,0,7.9250,True,False,False,False,True,False,False,False,False,False,False,False,False,True
3,1,35.0,1,0,53.1000,True,False,False,False,True,False,False,True,False,False,False,False,False,False
4,3,35.0,0,0,8.0500,False,True,False,False,True,False,False,False,False,False,False,False,False,True


In [21]:
X_test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin Index_A,Cabin Index_B,Cabin Index_C,Cabin Index_D,Cabin Index_E,Cabin Index_F,Cabin Index_G,Cabin Index_Unknown
0,3,34.5,0,0,7.8292,False,True,False,True,False,False,False,False,False,False,False,False,True
1,3,47.0,1,0,7.0000,True,False,False,False,True,False,False,False,False,False,False,False,True
2,2,62.0,0,0,9.6875,False,True,False,True,False,False,False,False,False,False,False,False,True
3,3,27.0,0,0,8.6625,False,True,False,False,True,False,False,False,False,False,False,False,True
4,3,22.0,1,1,12.2875,True,False,False,False,True,False,False,False,False,False,False,False,True


In [22]:
for col in X.columns:
    print(col)

Pclass
Age
SibSp
Parch
Fare
Sex_female
Sex_male
Embarked_C
Embarked_Q
Embarked_S
Cabin Index_A
Cabin Index_B
Cabin Index_C
Cabin Index_D
Cabin Index_E
Cabin Index_F
Cabin Index_G
Cabin Index_T
Cabin Index_Unknown


In [23]:
for col in X.columns:
    if col not in X_test.columns:
        X_test[col] = False

In [24]:
X_train = X.copy()
X_train = X_train.reindex(sorted(X_train.columns), axis=1)

In [25]:
X_train.head()

,Age,Cabin Index_A,Cabin Index_B,Cabin Index_C,Cabin Index_D,Cabin Index_E,Cabin Index_F,Cabin Index_G,Cabin Index_T,Cabin Index_Unknown,Embarked_C,Embarked_Q,Embarked_S,Fare,Parch,Pclass,Sex_female,Sex_male,SibSp
0,22.0,False,False,False,False,False,False,False,False,True,False,False,True,7.2500,0,3,False,True,1
1,38.0,False,False,True,False,False,False,False,False,False,True,False,False,71.2833,0,1,True,False,1
2,26.0,False,False,False,False,False,False,False,False,True,False,False,True,7.9250,0,3,True,False,0
3,35.0,False,False,True,False,False,False,False,False,False,False,False,True,53.1000,0,1,True,False,1
4,35.0,False,False,False,False,False,False,False,False,True,False,False,True,8.0500,0,3,False,True,0


In [26]:
X_pred = X_test.copy()
X_pred = X_pred.reindex(sorted(X_pred.columns), axis=1)

In [27]:
X_pred.head()

,Age,Cabin Index_A,Cabin Index_B,Cabin Index_C,Cabin Index_D,Cabin Index_E,Cabin Index_F,Cabin Index_G,Cabin Index_T,Cabin Index_Unknown,Embarked_C,Embarked_Q,Embarked_S,Fare,Parch,Pclass,Sex_female,Sex_male,SibSp
0,34.5,False,False,False,False,False,False,False,False,True,False,True,False,7.8292,0,3,False,True,0
1,47.0,False,False,False,False,False,False,False,False,True,False,False,True,7.0000,0,3,True,False,1
2,62.0,False,False,False,False,False,False,False,False,True,False,True,False,9.6875,0,2,False,True,0
3,27.0,False,False,False,False,False,False,False,False,True,False,False,True,8.6625,0,3,False,True,0
4,22.0,False,False,False,False,False,False,False,False,True,False,False,True,12.2875,1,3,True,False,1


In [28]:
# Secon Model XGBoost Regression
xgb_cla = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgb_cla.fit(X_train, y)
predictions = xgb_cla.predict(X_pred)
#xgb_mse = mean_squared_error(y_test, xgb_pred)
#xgb_training_time = time.time() - start_time

In [29]:
# First model : Random Forest
#model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
#model.fit(X_train, y)
#predictions = model.predict(X_pred)

In [30]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [31]:
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
